In [1]:
import functools
import os
import pickle
import random

import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels.formula.api as smf
from scipy.special import gamma


@functools.wraps(smf.ols)
def lm(*args, **kwargs):
    return smf.ols(*args, **kwargs).fit()

In [2]:
os.chdir("../data")

In [3]:
recommendee = "Fro116"
neighborhood_size = 8192
# recommendee = "mpfei"
# neighborhood_size = 724
confidence_interval = 0.99
normalize_variance = False

In [4]:
anime = pd.read_csv("AnimeList.csv")
anime = anime[["anime_id", "title", "type"]]

In [5]:
df = pd.read_csv("UserAnimeList.csv")

In [6]:
len(df["username"].unique()), len(df["anime_id"].unique())

(283045, 14478)

In [7]:
filtered_df = df[["username", "anime_id", "my_score"]].loc[lambda x: x["my_score"] != 0]

In [8]:
# add additional user anime-lists
extraUsers = pickle.load(open("user_profiles/ExtraUserAnimeLists.pkl", "rb"))
filtered_df = filtered_df.loc[lambda x: ~x["username"].isin(extraUsers.username)]
filtered_df = pd.concat([filtered_df, extraUsers], ignore_index=True)

In [9]:
average_rating = filtered_df["my_score"].mean()
user_bias = (
    pd.DataFrame(filtered_df.groupby("username")["my_score"].mean()).rename(
        {"my_score": "user_bias"}, axis=1
    )
    - average_rating
)
anime_bias = (
    pd.DataFrame(filtered_df.groupby("anime_id")["my_score"].mean()).rename(
        {"my_score": "anime_bias"}, axis=1
    )
    - average_rating
)

In [10]:
filtered_df = filtered_df.merge(anime_bias, on=["anime_id"]).merge(
    user_bias, on=["username"]
)
filtered_df["normalized_score"] = (
    filtered_df["my_score"]
    - filtered_df["anime_bias"]
    - filtered_df["user_bias"]
    - average_rating
)
filtered_df = filtered_df.set_index("username")
filtered_df = filtered_df.dropna()

In [11]:
filtered_df

,anime_id,my_score,anime_bias,user_bias,normalized_score
username,,,,,
karthiga,21,9,0.960563,-0.059899,0.605474
karthiga,59,7,0.040202,-0.059899,-0.474165
karthiga,74,7,0.316282,-0.059899,-0.750244
karthiga,120,7,0.309858,-0.059899,-0.743820
karthiga,178,7,-0.227339,-0.059899,-0.206623
...,...,...,...,...,...
temptemptemp,10040,6,-1.636718,-1.493861,1.636718
cinnamoroller,12963,10,-0.798861,2.506139,0.798861
inactiveX,5143,7,-0.652952,-0.493861,0.652952


In [12]:
if normalize_variance:
    user_stds = (
        filtered_df.groupby("username")[["normalized_score"]]
        .std()
        .rename({"normalized_score": "user_std"}, axis=1)
    )
    filtered_df = filtered_df.merge(user_stds, on="username")
    filtered_df["normalized_score"] /= filtered_df["user_std"]
    filtered_df = filtered_df.drop("user_std", axis=1)

In [13]:
user_subset = filtered_df.loc[[recommendee]].merge(
    filtered_df.reset_index(), on="anime_id"
)

In [14]:
adj_cos_corr_numerator = user_subset.groupby("username").apply(
    lambda x: np.dot(x["normalized_score_x"], x["normalized_score_y"])
)
adj_cos_corr_denom = filtered_df.groupby("username").apply(
    lambda x: np.sqrt(np.dot(x["normalized_score"], x["normalized_score"]))
)
adj_cos_corr_denom *= adj_cos_corr_denom.loc[recommendee]
adj_cos_corrs = pd.DataFrame(
    (adj_cos_corr_numerator / adj_cos_corr_denom), columns=["corr"]
)
adj_cos_corrs = adj_cos_corrs.dropna()

In [15]:
corrs = adj_cos_corrs.copy()
corrs["similarity"] = corrs["corr"].abs()
corrs["size"] = user_subset.groupby("username").size()
corrs = corrs.drop(
    recommendee
)  # Technically not needed because it's a noop for new series, but its useful for debugging

# We assume variance is the same as the variance for pearson correlation.
# see https://www.jstor.org/stable/2277400?seq=1
corrs = corrs.loc[lambda x: x["size"] > 2]
corrs["corr_var"] = (1 - corrs["corr"] * corrs["corr"]) ** 2 / (corrs["size"] - 2)
corrs = corrs.sort_values(by="similarity").dropna()[-neighborhood_size:]

In [16]:
corrs.describe()

,corr,similarity,size,corr_var
count,8192.000000,8192.000000,8192.000000,8192.000000
mean,0.041705,0.083595,42.776123,0.090556
std,0.074050,0.015295,35.347202,0.177023
min,-0.146716,0.067663,3.000000,0.003549
25%,-0.068180,0.072475,16.000000,0.016812
50%,0.074844,0.079080,34.000000,0.030860
75%,0.086498,0.089976,60.000000,0.070745
max,0.192775,0.192775,281.000000,0.990856


In [17]:
score = (filtered_df.merge(pd.DataFrame(corrs), on="username")).dropna()

user_var = (
    pd.DataFrame(filtered_df.groupby("username")["normalized_score"].var())
    .rename({"normalized_score": "user_var"}, axis=1)
    .dropna()
)
score = score.merge(user_var, on="username")

anime_var = (
    pd.DataFrame(filtered_df.groupby("anime_id")["normalized_score"].var())
    .rename({"normalized_score": "anime_var"}, axis=1)
    .dropna()
)
score = score.merge(anime_var, on="anime_id")

In [18]:
score

,anime_id,my_score,anime_bias,user_bias,normalized_score,corr,similarity,size,corr_var,user_var,anime_var
0,21,8,0.960563,0.520845,-0.975269,0.069141,0.069141,26,0.041269,1.355950,2.650513
1,21,10,0.960563,-0.271639,1.817214,-0.085849,0.085849,26,0.041055,5.162514,2.650513
2,21,10,0.960563,1.216665,0.328910,0.076163,0.076163,61,0.016753,0.629076,2.650513
3,21,8,0.960563,-0.970051,0.515627,0.094642,0.094642,10,0.122771,7.148232,2.650513
4,21,10,0.960563,0.481749,1.063827,0.081656,0.081656,20,0.054817,2.547335,2.650513
...,...,...,...,...,...,...,...,...,...,...,...
1231689,4332,10,-0.227194,1.654287,1.079046,-0.122185,0.122185,6,0.242591,5.846162,1.396115
1231690,12347,8,-0.972882,1.800257,-0.321236,0.078267,0.078267,44,0.023519,1.047412,1.510500
1231691,7216,5,-1.933861,0.238282,-0.798282,-0.070783,0.070783,21,0.052106,1.733848,1.220686
1231692,1745,6,-2.743861,-0.136718,1.386718,0.090471,0.090471,9,0.140528,3.249869,4.382262


In [19]:
deltas = score.groupby("anime_id").apply(
    lambda x: np.dot(x["normalized_score"], x["corr"]) / x["corr"].abs().sum()
)
weights = score.groupby("anime_id").apply(lambda x: x["corr"].abs().sum())
counts = score.groupby("anime_id").size()

In [20]:
# The following formulae are used to compute the variance of the delta. Delta
# is a weighted sum of the form δ = Σ(s_i * w_i) / (Σw_i), where s_i is
# a vector scores for user i and w_i is the weight for user_i.
#
# By linearity, it suffices to compute (s_i * w_i) / (Σw_i). We assume that
# Var(s_i) is the same as the variance over the vector s_i (i.e. over
# all items s_i has rated). We treat w_i as a random variable with mean w_i
# and variance corr['corr_var']
#
# The variance for (w_i) / (Σw_i) can be estimated by doing a Taylor Approximation.
# See equation 20 of https://www.stat.cmu.edu/~hseltman/files/ratio.pdf. The
# formula for the ratio of two correlated variables R,S is
# Var(R/S) = E[R]^2/E[S]^2(Var[R]/E[R]^2 - 2Cov(R,S)/(E[R]E[S]) + Var[S]/E[S]^2)
#
# Lastly we take the product distribution of s_i and (w_i) / (Σw_i).
def correction_factor(x):
    return (
        1
        + x["corr_var"] / (x["corr"] ** 2)
        - 2 * x["corr_var"] / (x["corr"].abs().sum() * x["corr"].abs())
        + x["corr_var"].sum() / (x["corr"].abs().sum() ** 2)
    )


delta_var = score.groupby("anime_id").apply(
    lambda x: np.sum(x["user_var"] * x["corr"] ** 2 * correction_factor(x))
    / (x["corr"].abs().sum() ** 2)
)

# if the var < 0, then the ratio distribution approximation failed,
# usually because sample size is too small
delta_var.loc[lambda x: x < 0] = np.inf

# Apply a bessel correction to unbias the variance
average_weight = corrs["corr"].abs().mean()
effective_sample_size = weights / average_weight
delta_var.loc[effective_sample_size <= 1] = np.inf
delta_var.loc[effective_sample_size > 1] *= effective_sample_size / (effective_sample_size - 1)

In [21]:
pred_df = pd.DataFrame()
pred_df["delta"] = deltas
pred_df["weight"] = weights
pred_df["counts"] = counts
pred_df["delta_sem"] = np.sqrt(delta_var)

# Unbias the standard deviation estimate
# See https://en.wikipedia.org/wiki/Unbiased_estimation_of_standard_deviation#Results_for_the_normal_distribution
def standard_deviation_bias(n):
    if n < 1:
        return np.inf
    if gamma((n - 1) / 2) == np.inf:
        return 1
    return np.sqrt(2 / (n - 1)) * gamma(n / 2) / gamma((n - 1) / 2)
pred_df["delta_sem"] /= effective_sample_size.apply(standard_deviation_bias)

pred_df["blp"] = anime_bias + user_bias.loc[recommendee].squeeze() + average_rating
pred_df = pred_df.dropna()

recomendee_seen_shows = filtered_df.loc[recommendee].merge(pred_df, on=["anime_id"])
recomendee_seen_shows["target"] = (
    recomendee_seen_shows["my_score"] - recomendee_seen_shows["blp"]
)
model = lm("target ~ delta + 0", recomendee_seen_shows)
pred_df["score"] = model.predict(pred_df) + pred_df["blp"]
pred_df["sem"] = np.sqrt(
    (
        (pred_df["delta_sem"] ** 2 + pred_df["delta"] ** 2)
        * (model.bse["delta"] ** 2 + model.params["delta"] ** 2)
    )
    - pred_df["delta"] ** 2 * model.params["delta"] ** 2
)

zscore = st.norm.ppf(1 - (1 - confidence_interval) / 2)
pred_df["score_lower_bound"] = pred_df["score"] - pred_df["sem"] * zscore
pred_df["score_upper_bound"] = pred_df["score"] + pred_df["sem"] * zscore

pred_df = pred_df.merge(anime, on="anime_id")
pred_df = pred_df.set_index("anime_id")

In [22]:
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:                 target   R-squared (uncentered):                   0.751
Model:                            OLS   Adj. R-squared (uncentered):              0.750
Method:                 Least Squares   F-statistic:                              1041.
Date:                Wed, 05 May 2021   Prob (F-statistic):                   2.33e-106
Time:                        00:16:15   Log-Likelihood:                         -425.50
No. Observations:                 347   AIC:                                      853.0
Df Residuals:                     346   BIC:                                      856.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [23]:
# confirm that setting blp = 1 is reasonable
print(lm("my_score ~ delta + blp + 0", recomendee_seen_shows).summary())

                                 OLS Regression Results                                
Dep. Variable:               my_score   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                          1.185e+04
Date:                Wed, 05 May 2021   Prob (F-statistic):                   1.06e-318
Time:                        00:16:16   Log-Likelihood:                         -416.80
No. Observations:                 347   AIC:                                      837.6
Df Residuals:                     345   BIC:                                      845.3
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [24]:
# confirm that the top shows are ones that the user rates highly
pred_df.loc[lambda x: (x["delta"] > 0)].sort_values(by="score_lower_bound", ascending=False)[:20]

,delta,weight,counts,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,
32,1.103561,209.455795,2491,0.065902,7.318568,10.475853,0.212503,9.928480,11.023226,Neon Genesis Evangelion: The End of Evangelion,Movie
2476,1.790678,192.030860,2318,0.083111,5.166755,10.289882,0.286012,9.553165,11.026599,School Days,TV
30,0.842739,296.125205,3543,0.057734,7.185697,9.596772,0.181366,9.129605,10.063938,Neon Genesis Evangelion,TV
3784,0.603789,165.688196,1969,0.065384,7.540291,9.267730,0.194661,8.766316,9.769143,Evangelion: 2.0 You Can (Not) Advance,Movie
16201,1.303122,53.520541,643,0.113405,5.708679,9.436908,0.344558,8.549386,10.324430,Aku no Hana,TV
3297,0.689486,29.345005,350,0.161015,7.645537,9.618154,0.464922,8.420594,10.815714,Aria The Origination,TV
11981,0.531959,106.821515,1276,0.076711,7.403852,8.925787,0.224585,8.347295,9.504279,Mahou Shoujo Madoka★Magica Movie 3: Hangyaku n...,Movie
820,0.525167,41.831643,503,0.146188,7.866787,9.369290,0.421026,8.284798,10.453781,Ginga Eiyuu Densetsu,OVA
9756,0.430506,291.625985,3505,0.056264,7.442672,8.674348,0.165509,8.248025,9.100671,Mahou Shoujo Madoka★Magica,TV


In [25]:
new_recs = pred_df.drop(filtered_df.loc[recommendee].anime_id, errors="ignore").loc[
    lambda x: (x["type"] != "Movie")
    & (x["type"] != "Special")
    & (x["type"] != "OVA")
    & (x["type"] != "ONA")
    & (x["type"] != "Music")
]

In [26]:
new_recs.loc[lambda x: (x["delta"] > 0)].sort_values(
    by="score_lower_bound", ascending=False
)[:20]

,delta,weight,counts,delta_sem,blp,score,sem,score_lower_bound,score_upper_bound,title,type
anime_id,,,,,,,,,,,
33089,0.490736,13.373087,162,0.196152,6.406620,7.810614,0.563142,6.360055,9.261173,Kemono Friends,TV
32681,0.331727,28.446973,343,0.135114,6.373492,7.322561,0.387862,6.323494,8.321628,Uchuu Patrol Luluco,TV
26,0.205859,39.260005,470,0.120574,6.508429,7.097392,0.345610,6.207160,7.987624,Texhnolyze,TV
26165,0.447544,20.913419,257,0.157229,5.889878,7.170301,0.451793,6.006558,8.334043,Yuri Kuma Arashi,TV
2403,0.330420,31.188409,385,0.141749,6.071557,7.016889,0.406795,5.969055,8.064724,Kodomo no Jikan (TV),TV
14131,0.088236,50.085262,609,0.115548,6.534439,6.786882,0.330835,5.934706,7.639057,Girls & Panzer,TV
3604,0.100457,13.130397,158,0.175077,6.864588,7.151995,0.501215,5.860950,8.443039,Hidamari Sketch x 365,TV
80,0.195442,20.480635,251,0.196825,6.747656,7.306815,0.563653,5.854942,8.758688,Mobile Suit Gundam,TV
85,0.150644,14.924625,184,0.202867,6.906227,7.337218,0.580834,5.841088,8.833347,Mobile Suit Zeta Gundam,TV


In [27]:
list(
    new_recs.loc[lambda x: (x["delta"] > 0)].sort_values(by="score_lower_bound")[-20:][
        "title"
    ]
)

['Love Live! Sunshine!! 2nd Season',
 'Hidamari Sketch x ☆☆☆',
 'Hidamari Sketch',
 'Koi Kaze',
 'Soredemo Machi wa Mawatteiru',
 'Tenkuu no Escaflowne',
 'Macross',
 'Kimi ga Nozomu Eien',
 'Kemonozume',
 'Mugen no Ryvius',
 'Space☆Dandy',
 'Mobile Suit Zeta Gundam',
 'Mobile Suit Gundam',
 'Hidamari Sketch x 365',
 'Girls & Panzer',
 'Kodomo no Jikan (TV)',
 'Yuri Kuma Arashi',
 'Texhnolyze',
 'Uchuu Patrol Luluco',
 'Kemono Friends']

In [28]:
pred_df.to_pickle("deltas/user.pkl")

In [29]:
filtered_df.loc[recommendee].to_pickle("deltas/recommendee.pkl")